In [3]:
import numpy as np
import time

# Чтение точек из файла
def read_points(file_name):
    data = np.loadtxt(file_name, dtype={'names': ('class_label', 'x', 'y', 'z'),
                                        'formats': ('U10', 'f4', 'f4', 'f4')})
    class_labels = data['class_label']
    coordinates = np.column_stack((data['x'], data['y'], data['z']))
    return class_labels, coordinates

# Проверка попадания точек в песочные часы
def points_in_hourglass(coords, apex, direction, cos_alpha, h):
    vec_to_points = coords - apex
    dist_to_apex = np.linalg.norm(vec_to_points, axis=1)

    dot_products = np.einsum('ij,j->i', vec_to_points, direction) / dist_to_apex

    in_cone = (np.abs(dot_products) >= cos_alpha) & (dist_to_apex <= h)
    return in_cone

# Генерация вращений направлений
def rotate_directions(initial_direction, angles_deg):
    angles_rad = np.radians(angles_deg, dtype=np.float32)
    cos_angles = np.cos(angles_rad)
    sin_angles = np.sin(angles_rad)

    x, y, z = initial_direction
    rotated = np.column_stack((
        x * cos_angles - y * sin_angles,
        x * sin_angles + y * cos_angles,
        np.full_like(cos_angles, z, dtype=np.float32)
    ))
    return rotated

# Фильтрация точек в песочных часах
def filter_points_in_hourglass(file_name, apex, initial_direction, alpha=90, h=1.0, target_count=10):
    class_labels, coords = read_points(file_name)
    cos_alpha = np.cos(np.radians(alpha), dtype=np.float32)
    selected_points = []

    angles = np.linspace(0, 360, num=360, endpoint=False, dtype=np.float32)
    rotated_directions = rotate_directions(initial_direction, angles)

    for direction in rotated_directions:
        in_hourglass = points_in_hourglass(coords, apex, direction, cos_alpha, h)
        indices = np.flatnonzero(in_hourglass)

        selected_points.extend([(class_labels[i], coords[i]) for i in indices])

        if len(selected_points) >= target_count:
            return selected_points[:target_count]

    return selected_points[:target_count]

if __name__ == "__main__":
    apex = np.array([0, 0, 0], dtype=np.float32)
    initial_direction = np.array([0, 0, 1], dtype=np.float32)

    start_time = time.time()
    selected_points = filter_points_in_hourglass('Niagara.xyz', apex, initial_direction)
    end_time = time.time()

    print(f"Время выполнения программы: {end_time - start_time:.6f} секунд")


Время выполнения программы: 68.771165 секунд
